<a href="https://colab.research.google.com/github/rija-ansari/MSE1003H_RijaAnsari/blob/main/MSE1003_Assignment1_RA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Introduction

1. Research the dataset for this project.
2. Define the research question

In [ ]:
import os

In [ ]:
from dotenv import load_dotenv
load_dotenv()
API_KEY = os.getenv('API_KEY')